In [1]:
# To Do: Create global config object for things that are not tied to a
#        particular dataset configuration but may still be changeable, e.g.
#        user messages related to input validation. Or maybe not this is an
#        interactive notebook not some compiled software.

#        Need exception handling, currently very optimistic

import ipywidgets as widgets
import json
import codecs
import datetime
import warnings

from IPython.display import display

In [2]:
# Load default dataset configuration object.
config = {}
with open('./configs/employees/default_employees.json') as default_configuration_json:
    config = json.load(default_configuration_json)

In [3]:
# Standard Properties
l3_box_layout = widgets.Layout(
      display         = 'flex'
    , flex_flow       = 'column'
    , align_content   = 'center'
    , justify_content = 'center'
)

l3_text_layout = widgets.Layout(
    display           = 'flex'
  , justify_content   = 'center'
  , width             = '25%'
)

l3_input_layout = widgets.Layout(
    width             = 'max_content'
)

l3_input_style = {
    'description_width': '50%'
}

In [4]:
# Standard Procedures & Globals
current_widget_id = 0
deleted_text_flag = '!DELETE ME!'
unassigned_option_list_value = 'unassigned'


# All options properties for widgets that use them are found in this dictionary.
# Some options dynamically update (e.g. when a new division is created it becames
# an option for departments).
# Some options are based on hierarchies, i.e. once a team has a division set
# only departments that sit under that division should be shown as valid options.
option_lists = {
    # Dynamic Lists
      'locations':   [unassigned_option_list_value] + [v['location_name']   for v in config['org_structure']['locations']['values']]
    , 'divisions':   [unassigned_option_list_value] + [v['division_name']   for v in config['org_structure']['divisions']['values']]
    , 'departments': [unassigned_option_list_value] + [v['department_name'] for v in config['org_structure']['departments']['values']]
    
    # Hierarchies
    , 'division_department_hiearchy': {unassigned_option_list_value: [unassigned_option_list_value]}
    
    # Static Lists (for now)
    , 'modifier_position':       ['pre', 'post', 'replace']
    , 'location_purpose':        ['Office', 'Manufacturing', 'Warehouse', 'Retail', 'Showroom']
    , 'location_occupancy_type': ['Owned', 'Leased']
}


def obtain_new_widget_id():
# For procedurally generated widgets assign them an ID so that we can safely go
# back and remove them later as required.
    global current_widget_id
    current_widget_id += 1
    return current_widget_id


def get_element_from_tooltip(tooltip, element):
# ipywidgets doesn't natively support parent/child awareness, working around this
# by embedding row details in the tooltips of widgets within that row as JSON.
    return json.loads('{' + tooltip + '}')[element]


def delete_input_group(parent, target_row_tooltip, child_index_with_tooltip_key, deleted_values):
# I haven't yet worked out how to safely delete a child widget from a parent.
# In the mean time we set input values to something that will be ignored when
# the output configuration file is generated and hide the inputs.

# Because we can't easily identify which row the delete button belongs to
# we need to search through all rows in the parent until we match on the
# tooltip of the delete button.
    for row_index, row in enumerate(ws[parent].children):
        if target_row_tooltip == row.children[child_index_with_tooltip_key].tooltip:
            for deleted_values_key, deleted_values_value in deleted_values.items():
                row.children[deleted_values_key].value = deleted_values_value

            # Closing the input objects seems safe it is just closing the object
            # that contains them, i.e. the row container that causes errors.
            for child_index, child in enumerate(row.children):
                child.close()
            
            # If we close out the container attempts to add new rows fail with error:
            # AttributeError: 'NoneType' object has no attribute 'comm_id'
            # row.close()
            
            row.layout.visibility = 'hidden'
            break


def add_entries_to_option_list(option_list_name, new_entries):
    global option_lists
    
    for new_entry in new_entries:
        if new_entry not in option_lists[option_list_name]:
            option_lists[option_list_name].append(new_entry)


def remove_entries_from_option_list(option_list_name, entries_to_remove):
    global option_lists
    
    for entry_to_remove in entries_to_remove:
        if entry_to_remove in option_lists[option_list_name]:
            option_lists[option_list_name].remove(entry_to_remove)
            
            
def add_entries_to_hiearchical_option_list(new_entries, hiearchy_name, hiearchy_parent):
    global option_lists
    
    for new_entry in new_entries:
        if new_entry not in option_lists[hiearchy_name][hiearchy_parent]:
            option_lists[hiearchy_name][hiearchy_parent].append(new_entry)
            

def remove_entries_from_hiearchical_option_list(entries_to_remove, hiearchy_name, hiearchy_parent):
    global option_lists
    
    for entry_to_remove in entries_to_remove:
        if entry_to_remove in option_lists[hiearchy_name][hiearchy_parent]:
            option_lists[hiearchy_name][hiearchy_parent].remove(entry_to_remove)
            

def find_first_parent_of_child_in_hiearchy(hiearchy_name, child):
    global option_lists
    
    first_parent = unassigned_option_list_value
    
    for parent, children in option_lists[hiearchy_name].items():
        if child in option_lists[hiearchy_name][parent]:
            first_parent = parent
            break
    
    return first_parent


def add_parent_to_hiearchy(hierarchy_name, parent):
    global option_lists
    
    if parent not in option_lists[hierarchy_name]:
        option_lists[hierarchy_name][parent] = [unassigned_option_list_value]


def remove_parent_from_hiearchy(hiearchy_name, parent):
    global option_lists
    
    del option_lists[hiearchy_name][parent]
            
            
def change_parent_name_in_hiearchy(hiearchy_name, old_parent_name, new_parent_name):
    global option_lists
    
    option_lists[hiearchy_name][new_parent_name] = option_lists[hiearchy_name][old_parent_name]
    remove_parent_from_hiearchy(
          hiearchy_name = hiearchy_name
        , parent        = old_parent_name
    )
    
    
def update_option_list_inputs(
      option_list_name
    , input_parent
    , input_index
    , replace_existing_with      = None
    , hiearchy_name              = None
    , hiearchy_parent            = None
    , set_existing_to_unassigned = False
):
# Standard function to change the optoisn value of widgets that use an option list.
# Required as we are constantly adjusting this property as related properties change
# e.g. when a new division is created the departmetn and team option lists need to
# be updated to allow its selection.
    global option_lists
    
    # It seems that changing the options list resets the value to the first option in the list.
    for group in ws[input_parent].children:
        current_value = group.children[input_index].value

        # Assume that if we can find the current value in the option list that
        # it has not been changed and should remain as is.
        # If we can't find the value any more check to see if an explicit
        # replacement value has been provided and if not set to unassigned.
        if unassigned_option_list_value:
            current_value = unassigned_option_list_value
        elif current_value in option_lists[option_list_name]:
            # No action required current value is not the one that changed.
            current_value = current_value
        elif replace_existing_with is not None:
            current_value = replace_existing_with
        else:
            current_vlaue = unassigned_option_list_value

        if hiearchy_name is None:
            group.children[input_index].options = option_lists[option_list_name]
        else:
            group.children[input_index].options = option_lists[hiearchy_name][hiearchy_parent]
        
        group.children[input_index].value = current_value

In [5]:
# Global Object to hold all widgets based on where they will be rendered.
# Makes organising/defining the visual layer a little neater rather than
# being littered throughout the notebook.
ws = {
      'l1_tab': widgets.Tab()

# Level 2: Overview ----------------------------------------------------------#
    , 'l2_overview':                           widgets.Box()
    , 'l3_dataset_details':                    widgets.Box(layout = l3_box_layout)
    , 'l3_dataset_details_load_configuration': widgets.FileUpload(
          description = 'Load Configuration'
        , accept      = 'application/json'
        , multiple    = False
    )
    , 'l3_dataset_details_save_configuration': widgets.Button(description = 'Save Configuration')

    , 'l3_dataset_details_configuration_name': widgets.Text(
          value       = ''
        , description = 'Configuration Name'
        , disabled    = False
        , layout      = l3_input_layout
        , style       = l3_input_style
    )

    , 'l3_dataset_details_dataset_configuration_last_modified': widgets.Text(
          value       = ''
        , description = 'Last Modified'
        , disabled    = True
        , layout      = l3_input_layout
        , style       = l3_input_style
    )

    , 'l3_dataset_details_dataset_name': widgets.Text(
          value       = ''
        , description = 'Dataset Name'
        , disabled    = True
        , layout      = l3_input_layout
        , style       = l3_input_style
    )
# ----------------------------------------------------------------------------#

# Level 2: Employee Properties -----------------------------------------------#
    , 'l2_employee_properties': widgets.Accordion()
    
    , 'l3_general_organisation_size': widgets.BoundedIntText(
          value       = 100
        , min         = 100
        , max         = 100000
        , description = 'Organisation Size'
        , disabled    = False
        , layout      = l3_input_layout
        , style       = l3_input_style
    )
    
    , 'l3_employment_type_mix':               widgets.Box(layout = l3_box_layout)
    , 'l3_employment_type_mix_header':        widgets.Box(layout = l3_box_layout)
    , 'l3_employment_type_mix_user_message':  widgets.Label(value = '')
    , 'l3_employment_type_mix_add_type':      widgets.Button(description = 'Add Type')    
    , 'l3_employment_type_mix_column_titles': widgets.HBox()
    , 'l3_employment_type_mix_total':         widgets.IntText(
              description = 'Total'
            , value       = 0
            , disabled    = True
          # , layout      = l3_input_layout
          # , style       = l3_input_style
        )
    , 'l3_employment_type_mix_body':          widgets.Box(layout = l3_box_layout)
    
    , 'l3_position_level_modifiers':               widgets.Box(layout = l3_box_layout)
    , 'l3_position_level_modifiers_add_modifier':  widgets.Button(description = 'Add Modifier')
    , 'l3_position_level_modifiers_column_titles': widgets.HBox()
    , 'l3_position_level_modifiers_body':          widgets.Box(layout = l3_box_layout)
# ----------------------------------------------------------------------------#

# Level 2: Org Structure -----------------------------------------------------#
    , 'l2_org_structure': widgets.Accordion()
    
    , 'l3_locations':                   widgets.Box(layout = l3_box_layout)
    , 'l3_locations_add_location':      widgets.Button(description = 'Add Location')
    , 'l3_locations_column_titles':     widgets.HBox()
    , 'l3_locations_body':              widgets.Box(layout = l3_box_layout)

    , 'l3_divisions':                   widgets.Box(layout = l3_box_layout)
    , 'l3_divisions_add_division':      widgets.Button(description = 'Add Division')
    , 'l3_divisions_column_titles':     widgets.HBox()
    , 'l3_divisions_body':              widgets.Box(layout = l3_box_layout)
    
    , 'l3_departments':                 widgets.Box(layout = l3_box_layout)
    , 'l3_departments_add_department':  widgets.Button(description = 'Add Department')
    , 'l3_departments_column_titles':   widgets.HBox()
    , 'l3_departments_body':            widgets.Box(layout = l3_box_layout)
# ----------------------------------------------------------------------------#
    
# Level 3: Teams -------------------------------------------------------------#
    , 'l2_teams':          widgets.VBox()
    , 'l3_teams_add_team': widgets.Button(description = 'Add Team')
    , 'l3_teams_body':     widgets.Accordion()
# ----------------------------------------------------------------------------#
}


# For dynamically created input groups record what index each group element is
# assigned. This helps stop cluttering up the application logic with human
# unfriendly indices and hopefully reduces the risk of typos.
input_indices = {
    'employment_type_mix': {
          'mix_name': 0
        , 'mix_percentage': 1
    },
    'position_level_modifiers': {
          'modifier_name': 0
        , 'modifier_position': 1
        , 'modifier_level': 2
    },
    'locations': {
          'location_name': 0
        , 'location_purpose': 1
        , 'location_occupancy_type': 2
    },
    'divisions': {
          'division_name': 0
        , 'division_leader': 1
        , 'division_leader_is_head_of_organisation': 2
    },
    'departments': {
          'department_name': 0
        , 'department_leader': 1
        , 'division': 2
    },
    'teams': {
          'team_name': 0
        , 'team_leader': 1
        , 'division': 2
        , 'department': 3
        , 'location': 4
        , 'titles': 5
        , 'level_1_percent': 6
        , 'level_2_percent': 7
        , 'level_3_percent': 8
    }
}

In [6]:
# Level 3: Overview / Dataset Details
# To Do: create validation process for dataset config.
def validate_configration(config_to_validate):
    return {'validation_passed': True}
    
    
def l3_overview_load_configuration(e):
# Use the FileUpload widget to enable the user to select a JSON file defining a
# dataset configuration that can be read by this tool. Config then gets loaded
# in and displayed.

# To Do: user feedback after load action i.e. was it successful or did it fail.

    global config
    config  = {}
    content = ws['l3_dataset_details_load_configuration'].value[0].content.tobytes()
    config  = json.loads(codecs.decode(content))
    
    if validate_configuration(config):
        initialise_all_values()


def l3_overview_save_configuration(e):
# Take the currently set values for the dataset configuration and save as a JSON file.
# To Do: user feedback after save action i.e. was it successful or did it fail, where did it save.
    now = datetime.datetime.now(datetime.timezone.utc)
    ws['l3_dataset_details_dataset_configuration_last_modified'].value = str(now)
    
    out_path = ''.join([
          './configs/employees/'
        , now.strftime('%Y%m%d_%H%M%S')
        , '_'
        , ws['l3_dataset_details_configuration_name'].value.strip().replace(' ', '_')
        , '.json'
    ])
    
    out_data = {
    "overview": {
        "dataset_details": {
            "configuration_name": ws['l3_dataset_details_configuration_name'].value,
            "last_modified": ws['l3_dataset_details_dataset_configuration_last_modified'].value,
            "dataset_name": "employees",
            "validation_passed": True,
            "validation_messages": []
        }
    },
    "employee_properties": {
        "organisation_size": {
            "description": "Organisation Size",
            "value": ws['l3_general_organisation_size'].value,
        },
        "employment_type_mix": {
            "description": "Employment Type Mix (percentage)",
            "values": l3_employment_type_mix_values_to_json()
            },
            "position_level_modifiers": {
                "description": "Position Level Modifers",
                "values": l3_position_level_modifiers_values_to_json()
            }
        },
        "org_structure": {
            "locations": {
                "description": "Locations",
                "values": l3_location_values_to_json()
            },
            "divisions": {
                "description": "Divisions",
                "values": l3_division_values_to_json()
            },
            "departments": {
                "description": "Departments",
                "values": l3_department_values_to_json()
            }
        },
        "teams": {
            "description": "Teams",
            "values": l3_team_values_to_json()
        }
    }
    
    if validate_configration(out_data):
        out_file = open(out_path, 'w')
        json.dump(out_data, out_file, indent = 2)
    

ws['l3_dataset_details_load_configuration'].observe(handler = l3_overview_load_configuration, names = 'value')
ws['l3_dataset_details_save_configuration'].on_click(l3_overview_save_configuration)
    
ws['l3_dataset_details'].children = [
      ws['l3_dataset_details_load_configuration']
    , ws['l3_dataset_details_save_configuration']
    , ws['l3_dataset_details_configuration_name']
    , ws['l3_dataset_details_dataset_configuration_last_modified']
    , ws['l3_dataset_details_dataset_name']
]

In [7]:
# Level 2: Overview
def l2_overview_initialise_values():
    ws['l3_dataset_details_configuration_name'].value                  = config['overview']['dataset_details']['configuration_name']
    ws['l3_dataset_details_dataset_configuration_last_modified'].value = config['overview']['dataset_details']['last_modified']
    ws['l3_dataset_details_dataset_name'].value                        = config['overview']['dataset_details']['dataset_name']


ws['l3_dataset_details_save_configuration'].on_click=(l3_overview_save_configuration)

# Maybe down the track we will want additional information in level 2 overview.
ws['l2_overview'] = ws['l3_dataset_details']

In [8]:
# Level 3: Employee Properties / Organisation Size
def l3_organisation_size_initialise_values():
    ws['l3_general_organisation_size'].value = config['employee_properties']['organisation_size']['value']

In [9]:
# Level 3: Employee Properties / Employment Type Mix
# To Do: Validate totals prior to generating JSON and throw warning if not 100%.
def l3_employment_type_mix_on_value_change(change):
# When an employment mix value is changed update the total and user feedback message.
    if change['old'] != change['new']:
        ws['l3_employment_type_mix_total'].value += change['new'] - change['old']

    if ws['l3_employment_type_mix_total'].value == 100:
        ws['l3_employment_type_mix_user_message'].value = 'Employment types total 100% Good to go!'
    elif ws['l3_employment_type_mix_total'].value > 100:
        ws['l3_employment_type_mix_user_message'].value = 'Warning: Employment types total over 100% Please correct mix.'
    else:
        ws['l3_employment_type_mix_user_message'].value = 'Warning: Employment types total under 100% Please correct mix.'


def l3_employment_type_mix_add(additional_mix_details_list = None):
# Take a list of employment type mixes and generate one input row per element.

# To Do: would be great to parametise this function based on the tooltip values
# but I'm not sure how to do that simply just yet.
# Could have a dictionary for:
#    default values
#    parents
#    also parametise the delete function
#    what is stumping me at the moment though is the dynamically handling the input rows

    # When a button is pressed the button object is passed to the function handler.
    # This checks for these instances and assumes that the button push was to add
    # a new entry and creates a list with default options.
    if additional_mix_details_list == None or additional_mix_details_list.__class__.__name__ == 'Button':
        additional_mix_details_list = [{
              'mix_name': 'enter employment type'
            , 'mix_percentage': 0
        }]
    
    additional_mix_list = []

    for index, row in enumerate(additional_mix_details_list):
        swap_widget_id = str(obtain_new_widget_id())
        tooltip_value  = '"Value": "' + row['mix_name'] + '", "ID": "' + swap_widget_id + '", "Parent": "l3_employment_type_mix_body"'
        
        additional_mix_list.append(widgets.HBox(children = [
              widgets.Text(value = row['mix_name'], continuous_update = False)
            , widgets.BoundedIntText(
                value       = row['mix_percentage'] * 100
              , min         = 0
              , max         = 100
              , step        = 1
            )
            , widgets.Button(description = 'Delete', tooltip = '"Action":"delete", ' + tooltip_value)
        ]))
        
    for row in additional_mix_list:
        row.children[input_indices['employment_type_mix']['mix_percentage']].observe(
              handler = l3_employment_type_mix_on_value_change
            , names   = 'value'
        )
        row.children[-1].on_click(l3_employment_type_mix_delete)

    ws['l3_employment_type_mix_body'].children += tuple(additional_mix_list)


def l3_employment_type_mix_delete(e):
    delete_input_group(
          parent                       = get_element_from_tooltip(tooltip = e.tooltip, element = 'Parent')
        , target_row_tooltip           = e.tooltip
        , child_index_with_tooltip_key = -1
        , deleted_values               = {
              input_indices['employment_type_mix']['mix_name']: deleted_text_flag
            , input_indices['employment_type_mix']['mix_percentage']: 0
        }
    )
    
    l3_employment_type_mix_on_value_change({'old': 0, 'new': 0})


def l3_employment_type_mix_values_to_json():
    return [
        {
              'mix_name': row.children[input_indices['employment_type_mix']['mix_name']].value.strip()
            , 'mix_percentage': row.children[input_indices['employment_type_mix']['mix_percentage']].value / 100
        }
        for row in ws['l3_employment_type_mix_body'].children
        if row.children[input_indices['employment_type_mix']['mix_name']].value != deleted_text_flag
    ]


def l3_employment_type_mix_initialise_values():
    ws['l3_employment_type_mix_body'].children = []
    l3_employment_type_mix_add(config['employee_properties']['employment_type_mix']['values'])

    ws['l3_employment_type_mix_total'].value = sum([
        row.children[input_indices['employment_type_mix']['mix_percentage']].value
        for row in ws['l3_employment_type_mix_body'].children
    ])
    
    l3_employment_type_mix_on_value_change({'old': 0, 'new': 0})


ws['l3_employment_type_mix_add_type'].on_click(l3_employment_type_mix_add)

ws['l3_employment_type_mix_column_titles'].children = [
      widgets.Text(value = 'Employment Type', disabled = True)
    , widgets.Text(value = 'Percentage'     , disabled = True)
    , widgets.Button(value = ''             , disabled = True, layout = {'visibility': 'hidden'})
]

ws['l3_employment_type_mix_header'].children = [
      ws['l3_employment_type_mix_user_message']
    , ws['l3_employment_type_mix_add_type']
    , ws['l3_employment_type_mix_total']
    , ws['l3_employment_type_mix_column_titles']
]

ws['l3_employment_type_mix'].children = [ws['l3_employment_type_mix_header'], ws['l3_employment_type_mix_body']]

In [10]:
# Level 3: Employee Properties / Position Level Modifiers
# To Do: 

def l3_position_level_modifiers_add(additional_modifier_details_list = None):
    if additional_modifier_details_list == None or additional_modifier_details_list.__class__.__name__ == 'Button':
        additional_modifier_details_list = [{
              'modifier_name': 'enter position modifier'
            , 'modifier_position': 'pre'
            , 'modifier_level': 0
        }]
    
    additional_modifier_list = []

    for index, row in enumerate(additional_modifier_details_list):
        swap_widget_id = str(obtain_new_widget_id())
        tooltip_value  = '"Value": "' + row['modifier_position'] + '", "ID": "' + swap_widget_id + '", "Parent": "l3_position_level_modifiers_body"'
        
        additional_modifier_list.append(widgets.HBox(children = [
              widgets.Text(    value = row['modifier_name'], continuous_update = False)
            , widgets.Dropdown(value = row['modifier_position'], options = option_lists['modifier_position'])
            , widgets.BoundedIntText(
                value       = row['modifier_level']
              , min         = 1
              , max         = 3
              , step        = 1
            )
            , widgets.Button(description = 'Delete', tooltip = '"Action": "delete", ' + tooltip_value)
        ]))
        
    for row in additional_modifier_list:
        row.children[-1].on_click(l3_position_level_modifiers_delete)

    ws['l3_position_level_modifiers_body'].children += tuple(additional_modifier_list)
    

def l3_position_level_modifiers_delete(e):
    delete_input_group(
          parent                       = get_element_from_tooltip(tooltip = e.tooltip, element = 'Parent')
        , target_row_tooltip           = e.tooltip
        , child_index_with_tooltip_key = -1
        , deleted_values               = {input_indices['position_level_modifiers']['modifier_name']: deleted_text_flag}
    )
    

def l3_position_level_modifiers_values_to_json():
    return [
        {
              'modifier_name':     row.children[input_indices['position_level_modifiers']['modifier_name']].value.strip()
            , 'modifier_position': row.children[input_indices['position_level_modifiers']['modifier_position']].value
            , 'modifier_level':    row.children[input_indices['position_level_modifiers']['modifier_level']].value
        }
        for row in ws['l3_position_level_modifiers_body'].children
        if row.children[input_indices['position_level_modifiers']['modifier_name']].value != deleted_text_flag
    ]
    

def l3_position_modifiers_initialise_values():
    ws['l3_position_level_modifiers_body'].children = []
    l3_position_level_modifiers_add(config['employee_properties']['position_level_modifiers']['values'])
    

ws['l3_position_level_modifiers_add_modifier'].on_click(l3_position_level_modifiers_add)

ws['l3_position_level_modifiers_column_titles'].children = [
      widgets.Text(value = 'Modifier Name'    , disabled = True)
    , widgets.Text(value = 'Modifier Position', disabled = True)
    , widgets.Text(value = 'Modifier Level'   , disabled = True)
    , widgets.Button(value = ''               , disabled = True, layout = {'visibility': 'hidden'})
]

ws['l3_position_level_modifiers'].children = [
      ws['l3_position_level_modifiers_add_modifier']
    , ws['l3_position_level_modifiers_column_titles']
    , ws['l3_position_level_modifiers_body']
]

In [11]:
# Level 2: Employee Properties
def l2_employee_properties_initialise_values():
    l3_organisation_size_initialise_values()
    l3_employment_type_mix_initialise_values()
    l3_position_modifiers_initialise_values()


ws['l2_employee_properties'].children = [
      ws['l3_general_organisation_size']
    , ws['l3_employment_type_mix']
    , ws['l3_position_level_modifiers']
]

ws['l2_employee_properties'].titles = [
      'Organisation Size'
    , 'Employment Type Mix (percentage)'
    , 'Position Level Modifiers'
]

In [12]:
# Level 3: Org Structure / Locations
def l3_locations_add(additional_location_details_list = None):
    if additional_location_details_list == None or additional_location_details_list.__class__.__name__ == 'Button':
        additional_location_details_list = [{
              'location_name': 'enter location name'
            , 'location_purpose': 'Office'
            , 'location_occupancy_type': 'Leased'
        }]
    
    additional_location_list = []

    for index, row in enumerate(additional_location_details_list):
        add_entries_to_option_list(option_list_name = 'locations', new_entries = [row['location_name']])
        
        swap_widget_id = str(obtain_new_widget_id())
        tooltip_value  = '"Value": "' + row['location_name'] + '", "ID": "' + swap_widget_id + '", "Parent": "l3_locations_body"'
        
        additional_location_list.append(widgets.HBox(children = [
              widgets.Text(    value = row['location_name'], continuous_update = False)
            , widgets.Dropdown(value = row['location_purpose']       , options = option_lists['location_purpose'])
            , widgets.Dropdown(value = row['location_occupancy_type'], options = option_lists['location_occupancy_type'])
            , widgets.Button(description = 'Delete', tooltip = '"Action": "delete", ' + tooltip_value)
        ]))
        
    for row in additional_location_list:
        row.children[input_indices['locations']['location_name']].observe(handler = l3_locations_change, names = 'value')
        row.children[-1].on_click(l3_locations_delete)

    ws['l3_locations_body'].children += tuple(additional_location_list)
    

def l3_locations_change(e):
    remove_entries_from_option_list(
          option_list_name  = 'locations'
        , entries_to_remove = [e['old']]
    )
    
    add_entries_to_option_list(
          option_list_name = 'locations'
        , new_entries      = [e['new']]
    )
    
    update_option_list_inputs(
          option_list_name      = 'locations'
        , input_parent          = 'l3_teams_body'
        , input_index           = input_indices['teams']['location']
        , replace_existing_with = e['new']
    )
    
    
def l3_locations_delete(e):
    remove_entries_from_option_list(
          option_list_name  = 'locations'
        , entries_to_remove = [get_element_from_tooltip(tooltip = e.tooltip, element = 'Value')]
    )
    
    update_option_list_inputs(
          option_list_name = 'locations'
        , input_parent     = 'l3_teams_body'
        , input_index      = input_indices['teams']['location']
    )
    
    delete_input_group(
          parent                       = get_element_from_tooltip(tooltip = e.tooltip, element = 'Parent')
        , target_row_tooltip           = e.tooltip
        , child_index_with_tooltip_key = -1
        , deleted_values               = {input_indices['locations']['location_name']: deleted_text_flag}
    )
    

def l3_location_values_to_json():
    return [
        {
              'location_name':           row.children[input_indices['locations']['location_name']].value.strip()
            , 'location_purpose':        row.children[input_indices['locations']['location_purpose']].value
            , 'location_occupancy_type': row.children[input_indices['locations']['location_occupancy_type']].value
        }
        for row in ws['l3_locations_body'].children
        if row.children[input_indices['locations']['location_name']].value != deleted_text_flag
    ]
    

def l3_locations_initialise_values():   
    ws['l3_locations_body'].children = []
    l3_locations_add(config['org_structure']['locations']['values'])
    

ws['l3_locations_add_location'].on_click(l3_locations_add)

ws['l3_locations_column_titles'].children = [
      widgets.Text(value = 'Location Name'           , disabled = True)
    , widgets.Text(value = 'Location Purpose'        , disabled = True)
    , widgets.Text(value = 'Location Occupancy Type' , disabled = True)
    , widgets.Button(value = ''                      , disabled = True, layout = {'visibility': 'hidden'})
]

ws['l3_locations'].children = [
      ws['l3_locations_add_location']
    , ws['l3_locations_column_titles']
    , ws['l3_locations_body']
]

In [13]:
# Level 3: Org Structure / Divisions
# To Do: Validate on initial setup that only one division head is flagged as org leader.
#        If the currently checked row is deleted make the first row checked
def l3_divisions_add(additional_division_details_list = None): 
    if additional_division_details_list == None or additional_division_details_list.__class__.__name__ == 'Button':
        additional_division_details_list = [{
              'division_name': 'enter division name'
            , 'division_leader': 'enter division leader'
            , 'division_leader_is_head_of_organisation': False
        }]
    
    additional_division_list = []

    for index, row in enumerate(additional_division_details_list):
        add_entries_to_option_list(option_list_name = 'divisions', new_entries = [row['division_name']])
        add_parent_to_hiearchy(hierarchy_name = 'division_department_hiearchy', parent = row['division_name'])
        
        swap_widget_id = str(obtain_new_widget_id())
        tooltip_value  = '"Value": "' + row['division_name'] + '", "ID": "' + swap_widget_id + '", "Parent": "l3_divisions_body"'
        
        additional_division_list.append(widgets.HBox(children = [
              widgets.Text(    value = row['division_name']  , continuous_update = False)
            , widgets.Text(    value = row['division_leader'], continuous_update = False)
            , widgets.Checkbox(value = row['division_leader_is_head_of_organisation'], tooltip = '"Action": "change", ' + tooltip_value)
            , widgets.Button(description = 'Delete', tooltip = '"Action": "delete", ' + tooltip_value)
        ]))
        
    for row in additional_division_list:
        row.children[input_indices['divisions']['division_name']].observe(
              handler = l3_divisions_change
            , names   = 'value'
        )
        row.children[input_indices['divisions']['division_leader_is_head_of_organisation']].observe(
              handler = l3_division_leader_is_head_of_org_change
            , names   = 'value'
        )
        row.children[-1].on_click(l3_divisions_delete)
        
    update_option_list_inputs(
          option_list_name = 'divisions'
        , input_parent     = 'l3_departments_body'
        , input_index      = input_indices['departments']['division']
    )
    
    update_option_list_inputs(
          option_list_name = 'divisions'
        , input_parent     = 'l3_teams_body'
        , input_index      = input_indices['teams']['division']
    )
        
    ws['l3_divisions_body'].children += tuple(additional_division_list)
    

def l3_divisions_change(e):
    remove_entries_from_option_list(
          option_list_name  = 'divisions'
        , entries_to_remove = [e['old']]
    )
    
    add_entries_to_option_list(
          option_list_name = 'divisions'
        , new_entries      = [e['new']]
    )
    
    change_parent_name_in_hiearchy(
          hiearchy_name   = 'division_department_hiearchy'
        , old_parent_name = e['old']
        , new_parent_name = e['new']
    )
    
    update_option_list_inputs(
          option_list_name      = 'divisions'
        , input_parent          = 'l3_departments_body'
        , input_index           = input_indices['departments']['division']
        , replace_existing_with = e['new']
    )
    
    update_option_list_inputs(
          option_list_name      = 'divisions'
        , input_parent          = 'l3_teams_body'
        , input_index           = input_indices['teams']['division']
        , replace_existing_with = e['new']
    )
    
    
def l3_divisions_delete(e):
    division_to_delete = get_element_from_tooltip(tooltip = e.tooltip, element = 'Value')
    
    remove_entries_from_option_list(
          option_list_name  = 'divisions'
        , entries_to_remove = [division_to_delete]
    )
    
    update_option_list_inputs(
          option_list_name = 'divisions'
        , input_parent     = 'l3_departments_body'
        , input_index      = input_indices['departments']['division']
    )
    
    update_option_list_inputs(
          option_list_name = 'divisions'
        , input_parent     = 'l3_teams_body'
        , input_index      = input_indices['teams']['division']
    )
    
    remove_parent_from_hiearchy(
          hiearchy_name = 'division_department_hiearchy'
        , parent        = division_to_delete
    )
        
    delete_input_group(
          parent                       = get_element_from_tooltip(tooltip = e.tooltip, element = 'Parent')
        , target_row_tooltip           = e.tooltip
        , child_index_with_tooltip_key = -1
        # If the deleted row has the division leader is head or org checked (index = 2) uncheck it.
        , deleted_values               = {
              input_indices['divisions']['division_name']: deleted_text_flag
            , input_indices['divisions']['division_leader_is_head_of_organisation']: False
        }
    )

    
def l3_division_leader_is_head_of_org_change(e):
    checkbox_index = input_indices['divisions']['division_leader_is_head_of_organisation']
    
    # When the value of a checkbox is set to True it is disabled so we can
    # ignore all changes where the new value is False. This is required
    # because I haven't found a way to set the value of the previously True
    # checkbox to False without triggering the handler. I have tried removing
    # all handlers before changing values but I couldn't find how to do this.   
    if e['new']:
        for row_index, row in enumerate(ws['l3_divisions_body'].children):

            # We have found the row that is now checked.
            if e.owner.tooltip == row.children[checkbox_index].tooltip:
                row.children[checkbox_index].disabled = True
            # We have found the row that was previously checked.
            elif row.children[checkbox_index].value:
                row.children[checkbox_index].disabled = False
                row.children[checkbox_index].value    = False


def l3_division_values_to_json():
    return [
        {
              'division_name':                           row.children[input_indices['divisions']['division_name']].value.strip()
            , 'division_leader':                         row.children[input_indices['divisions']['division_leader']].value.strip()
            , 'division_leader_is_head_of_organisation': row.children[input_indices['divisions']['division_leader_is_head_of_organisation']].value
        }
        for row in ws['l3_divisions_body'].children
        if row.children[input_indices['divisions']['division_name']].value != deleted_text_flag
    ]
    

def l3_divisions_initialise_values():
    ws['l3_divisions_body'].children = []
    l3_divisions_add(config['org_structure']['divisions']['values'])
    

ws['l3_divisions_add_division'].on_click(l3_divisions_add)

ws['l3_divisions_column_titles'].children = [
      widgets.Text(value = 'Division Name'                  , disabled = True)
    , widgets.Text(value = 'Division Leader'                , disabled = True)
    , widgets.Text(value = 'Division Leader Is Head of Org' , disabled = True)
    , widgets.Button(value = ''                             , disabled = True, layout = {'visibility': 'hidden'})
]

ws['l3_divisions'].children = [
      ws['l3_divisions_add_division']
    , ws['l3_divisions_column_titles']
    , ws['l3_divisions_body']
]

In [14]:
# Level 3: Org Structure / departments
def l3_departments_add(additional_department_details_list = None):
    if additional_department_details_list == None or additional_department_details_list.__class__.__name__ == 'Button':
        additional_department_details_list = [{
              'department_name':   'enter department name'
            , 'department_leader': 'Manager'
            , 'division':          option_lists['divisions'][0]
        }]
    
    additional_department_list = []

    for index, row in enumerate(additional_department_details_list):
        add_entries_to_option_list(
              option_list_name = 'departments'
            , new_entries      = [row['department_name']]
        )
        
        add_entries_to_hiearchical_option_list(
              new_entries     = [row['department_name']]
            , hiearchy_name   = 'division_department_hiearchy'
            , hiearchy_parent = row['division']
        )
        
        swap_widget_id = str(obtain_new_widget_id())
        tooltip_value  = '"Value": "' + row['department_name'] + '", "ID": "' + swap_widget_id + '", "Parent": "l3_departments_body"'
        
        additional_department_list.append(widgets.HBox(children = [
              widgets.Text(    value = row['department_name']  , continuous_update = False)
            , widgets.Text(    value = row['department_leader'], continuous_update = False)
            , widgets.Dropdown(value = row['division']         , options = option_lists['divisions'])
            , widgets.Button(description = 'Delete', tooltip = '"Action": "delete", ' + tooltip_value)
        ]))
        
    for row in additional_department_list:
        row.children[input_indices['departments']['department_name']].observe(handler = l3_departments_change, names = 'value')
        row.children[-1].on_click(l3_departments_delete)

    ws['l3_departments_body'].children += tuple(additional_department_list)
    

def l3_departments_change(e):
    division = find_first_parent_of_child_in_hiearchy(
          hiearchy_name = 'division_department_hiearchy'
        , child         = e['old']
    )
    
    remove_entries_from_option_list(
          option_list_name  = 'departments'
        , entries_to_remove = [e['old']]
    )
    
    add_entries_to_option_list(
          option_list_name = 'departments'
        , new_entries      = [e['new']]
    )
    
    add_entries_to_hiearchical_option_list(
          new_entries     = [e['new']]
        , hiearchy_name   = 'division_department_hiearchy'
        , hiearchy_parent = division
    )
    
    remove_entries_from_hiearchical_option_list(
          entries_to_remove = [e['old']]
        , hiearchy_name     = 'division_department_hiearchy'
        , hiearchy_parent   = division
    )
       
    update_option_list_inputs(
          option_list_name      = 'departments'
        , input_parent          = 'l3_teams_body'
        , input_index           = input_indices['teams']['department']
        , replace_existing_with = e['new']
        , hiearchy_name         = 'division_department_hiearchy'
        , hiearchy_parent       = division
    )
    
    
def l3_departments_delete(e):
    department_to_delete = get_element_from_tooltip(tooltip = e.tooltip, element = 'Value')
    
    division = find_first_parent_of_child_in_hiearchy(
          hiearchy_name = 'division_department_hiearchy'
        , child         = department_to_delete
    )
    
    remove_entries_from_option_list(
          option_list_name  = 'departments'
        , entries_to_remove = [department_to_delete]
    )
    
    remove_entries_from_hiearchical_option_list(
          entries_to_remove = [department_to_delete]
        , hiearchy_name     = 'division_department_hiearchy'
        , hiearchy_parent   = division
    )
    
    update_option_list_inputs(
          option_list_name = 'departments'
        , input_parent     = 'l3_teams_body'
        , input_index      = input_indices['teams']['department']
        , hiearchy_name    = 'division_department_hiearchy'
        , hiearchy_parent  = division
    )
    
    delete_input_group(
          input_parent                 = get_element_from_tooltip(tooltip = e.tooltip, element = 'Parent')
        , target_row_tooltip           = e.tooltip
        , child_index_with_tooltip_key = -1
        , deleted_values               = {input_indices['departments']['department_name']: deleted_text_flag}
    )
    

def l3_department_values_to_json():
    return [
        {
              'department_name':   row.children[input_indices['departments']['department_name']].value.strip()
            , 'department_leader': row.children[input_indices['departments']['department_leader']].value.strip()
            , 'division':          row.children[input_indices['departments']['division']].value
        }
        for row in ws['l3_departments_body'].children
        if row.children[input_indices['departments']['department_name']].value != deleted_text_flag
    ]
    

def l3_departments_initialise_values():
    ws['l3_departments_body'].children = []
    l3_departments_add(config['org_structure']['departments']['values'])
    

ws['l3_departments_add_department'].on_click(l3_departments_add)

ws['l3_departments_column_titles'].children = [
      widgets.Text(value = 'Department Name'  , disabled = True)
    , widgets.Text(value = 'Department Leader', disabled = True)
    , widgets.Text(value = 'Division'         , disabled = True)
    , widgets.Button(value = ''               , disabled = True, layout = {'visibility': 'hidden'})
]

ws['l3_departments'].children = [
      ws['l3_departments_add_department']
    , ws['l3_departments_column_titles']
    , ws['l3_departments_body']
]

In [15]:
# Level 2: Org Structure
def l2_org_structure_initialise_values():
    l3_locations_initialise_values()
    l3_divisions_initialise_values()
    l3_departments_initialise_values()
    

ws['l2_org_structure'].children = [
      ws['l3_locations']
    , ws['l3_divisions']
    , ws['l3_departments']
]

ws['l2_org_structure'].titles = [
      'Locations'
    , 'Divisions'
    , 'Departments'
]

In [16]:
# Level 2: Teams
def l3_teams_add(additional_team_details_list = None):
    if additional_team_details_list == None or additional_team_details_list.__class__.__name__ == 'Button':
        additional_team_details_list = [{
              'team_name':       'enter team name'
            , 'team_leader':     'Team Leader'
            , 'division':        option_lists['divisions'][0]
            , 'department':      option_lists['departments'][0]
            , 'location':        option_lists['locations'][0]
            , 'titles':          ['enter titles', 'as a comma (,)', 'separated list']
            , 'level_1_percent': 0.25
            , 'level_2_percent': 0.5
            , 'level_3_percent': 0.25
        }]
    
    additional_team_list = []

    for index, row in enumerate(additional_team_details_list):
        swap_widget_id = str(obtain_new_widget_id())
        tooltip_value  = '"Value": "' + row['team_name'] + '", "ID": "' + swap_widget_id + '", "Parent": "l3_teams_body"'
        
        additional_team_list.append(widgets.VBox(children = [
              widgets.Text(    value = row['team_name']        , description = 'Team Name'  , continuous_update = False)
            , widgets.Text(    value = row['team_leader']      , description = 'Team Leader', continuous_update = False)
            , widgets.Dropdown(value = row['division']         , description = 'Division'   , options = option_lists['divisions']  , tooltip = tooltip_value)
            , widgets.Dropdown(value = row['department']       , description = 'Department' , options = option_lists['departments'], tooltip = tooltip_value)
            , widgets.Dropdown(value = row['location']         , description = 'Location'   , options = option_lists['locations']  , tooltip = tooltip_value)
            , widgets.Text(    value = ', '.join(row['titles']), description = 'Titles'     , continuous_update = False)
            , widgets.BoundedIntText(
                value       = row['level_1_percent'] * 100
              , description = 'Level 1 %'
              , min         = 0
              , max         = 100
              , step        = 1
            )
            , widgets.BoundedIntText(
                value       = row['level_2_percent'] * 100
              , description = 'Level 2 %'
              , min         = 0
              , max         = 100
              , step        = 1
            )
            , widgets.BoundedIntText(
                value       = row['level_3_percent'] * 100
              , description = 'Level 3 %'
              , min         = 0
              , max         = 100
              , step        = 1
            )
            , widgets.Button(description = 'Delete', tooltip = '"Action": "delete", ' + tooltip_value)
        ]))
        
    for row in additional_team_list:
        row.children[input_indices['teams']['division']].observe(handler = l3_teams_division_change, names = 'value')
        row.children[-1].on_click(l3_teams_delete)

    ws['l3_teams_body'].children += tuple(additional_team_list)
    ws['l3_teams_body'].titles   = [team.children[input_indices['teams']['team_name']].value for team in ws['l3_teams_body'].children]
    

def l3_teams_division_change(e):
    update_option_list_inputs(
          option_list_name           = 'departments'
        , input_parent               = 'l3_teams_body'
        , input_index                = input_indices['teams']['department']
        , hiearchy_name              = 'division_department_hiearchy'
        , hiearchy_parent            = e['new']
        , set_existing_to_unassigned = True
    )
    

def l3_teams_delete(e):
    delete_input_group(
          parent                       = get_element_from_tooltip(tooltip = e.tooltip, element = 'Parent')
        , target_row_tooltip           = e.tooltip
        , child_index_with_tooltip_key = -1
        , deleted_values               = {input_indices['teams']['team_name']: deleted_text_flag}
    )
    

def l3_team_values_to_json():
    return [
        {
              'team_name':       row.children[input_indices['teams']['team_name']].value.strip()
            , 'team_leader':     row.children[input_indices['teams']['team_leader']].value.strip()
            , 'division':        row.children[input_indices['teams']['division']].value
            , 'department':      row.children[input_indices['teams']['department']].value
            , 'location':        row.children[input_indices['teams']['location']].value
            , 'titles':          [title.strip() for title in row.children[input_indices['teams']['titles']].value.split(',')]
            , 'level_1_percent': row.children[input_indices['teams']['level_1_percent']].value
            , 'level_2_percent': row.children[input_indices['teams']['level_2_percent']].value
            , 'level_3_percent': row.children[input_indices['teams']['level_3_percent']].value
        }
        for row in ws['l3_teams_body'].children
        if row.children[input_indices['teams']['team_name']].value != deleted_text_flag
    ]
    

def l3_teams_initialise_values():
    ws['l3_teams_body'].children = []
    l3_teams_add(config['teams']['values'])
    

ws['l3_teams_add_team'].on_click(l3_teams_add)
ws['l2_teams'].children = [ws['l3_teams_add_team'], ws['l3_teams_body']]

In [17]:
def initialise_all_values():
    l2_overview_initialise_values()
    l2_employee_properties_initialise_values()
    l2_org_structure_initialise_values()
    l3_teams_initialise_values()


ws['l1_tab'].children = [
      ws['l2_overview']
    , ws['l2_employee_properties']
    , ws['l2_org_structure']
    , ws['l2_teams']
]

ws['l1_tab'].titles = [
      'Overview'
    , 'Employee Properties'
    , 'Org Structure'
    , 'Teams'
  ]

initialise_all_values()
display(ws['l1_tab'])